# MCP Server vs Function Calling Demo

**Thesis:** "MCP server ≠ Function Calling and is NOT only used in case of AI"

This notebook demonstrates three different approaches:

1. **Simple MCP Client** - Non-AI application using MCP protocol
2. **AI + MCP Client** - AI application using MCP tools via protocol
3. **Traditional Function Calling** - AI application using direct function calls from MCP server

## Key Differences:
- **MCP Protocol**: Standardized tool discovery and calling over network
- **Function Calling**: Direct in-process function execution with AI
- **MCP enables non-AI apps** to use the same tools that AI can access!

## 1. Start MCP Server in Background

In [ ]:
import subprocess
import time
import requests
from mcp_server import stop_mcp_server

def start_new_mcp_server():
    """Start a new MCP server"""
    print("🚀 Starting new MCP server...")
    server_process = subprocess.Popen(
        ["python", "mcp_server.py", "sse", "8765"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    
    # Wait for server to start
    print("⏳ Waiting for server to start...")
    time.sleep(3)
    
    # Check if server is running
    max_retries = 5
    for retry in range(max_retries):
        try:
            response = requests.get("http://localhost:8765/health", timeout=2)
            if response.status_code == 200:
                print("✅ MCP server is running on http://localhost:8765")
                print(f"📊 Server PID: {server_process.pid}")
                return server_process
        except requests.RequestException:
            if retry < max_retries - 1:
                print(f"⏳ Retry {retry + 1}/{max_retries}...")
                time.sleep(2)
            else:
                print("⚠️  Server might still be starting... continuing anyway")
                print(f"📊 Server PID: {server_process.pid}")
                return server_process
    
    return server_process

# Main execution
print("🔧 MCP SERVER STARTUP PROCESS")
print("=" * 35)

# Step 1: Stop any existing MCP servers using utility from mcp_server.py
stop_mcp_server()

# Step 2: Start new MCP server
server_process = start_new_mcp_server()

print("\n✅ MCP server startup completed!")
print(f"🌐 Server available at: http://localhost:8765")
print(f"📊 Process ID: {server_process.pid}")

## 1.1 Stop MCP Server (if needed)

Run this cell if you need to stop the MCP server manually.

In [ ]:
from mcp_server import stop_mcp_server

# Execute the stop function from mcp_server.py
stop_mcp_server()

## 2. Simple MCP Client (Non-AI Application)

This demonstrates that **MCP is NOT just for AI** - any application can use MCP protocol to discover and call tools.

### 2.1 Transport Options Example

Your MCP server supports different transport methods. Here are the client configurations:

**Current Setup (SSE):**
```python
# Server: python mcp_server.py sse 8765
client = Client("http://localhost:8765/sse")
```

**Alternative Transports:**
```python
# HTTP: python mcp_server.py http 8765
client = Client("http://localhost:8765")

# Streamable HTTP: python mcp_server.py streamable-http 8765  
client = Client("http://localhost:8765/stream")

# STDIO: python mcp_server.py stdio
# For STDIO, you can use simple file path syntax:
client = Client("mcp_server.py")  # FastMCP automatically handles subprocess

# Or explicit subprocess control (for advanced configuration):
import subprocess
from fastmcp.client.transports import StdioTransport

transport = StdioTransport(
    command="python",
    args=["mcp_server.py", "stdio"]
)
client = Client(transport)
```

In [ ]:
from fastmcp import Client
import json

async def simple_mcp_client_demo():
    """Demonstrate non-AI application using MCP protocol"""
    print("🔍 SIMPLE MCP CLIENT DEMO (Non-AI Application)")
    print("=" * 50)
    
    # MCP Client URL Options based on server transport:
    # - SSE (current):           "http://localhost:8765/sse"
    # - HTTP:                    "http://localhost:8765"
    # - Streamable HTTP:         "http://localhost:8765/stream"
    # - STDIO:                   Client("mcp_server.py") - simple file path syntax
    client = Client("http://localhost:8765/sse")
    async with client:
        # 1. Server Health Check
        print("\n🔗 SERVER CONNECTIVITY:")
        print("=" * 25)
        try:
            ping_result = await client.ping()
            print("✅ Server ping successful!")
        except Exception as e:
            print(f"❌ Server ping failed: {e}")
        
        # 2. Server Information
        print("\n📊 SERVER INFORMATION:")
        print("=" * 25)
        try:
            server_info = await client.get_server_info()
            print("✅ Server Info:")
            print(json.dumps(server_info, indent=2))
        except Exception as e:
            print(f"⚠️ Server info not available: {e}")
        
        try:
            capabilities = await client.get_server_capabilities()
            print("\n✅ Server Capabilities:")
            print(json.dumps(capabilities, indent=2))
        except Exception as e:
            print(f"⚠️ Server capabilities not available: {e}")
        
        # 3. Complete Discovery via MCP Protocol
        print("\n📋 DISCOVERY - ALL AVAILABLE FEATURES:")
        print("=" * 45)
        
        # Tools discovery
        print("\n🛠️ Available Tools:")
        tools = await client.list_tools()
        print(f"Found {len(tools)} tools:")
        for tool in tools:
            print(f"  • {tool.name}: {tool.description}")
        
        # Resources discovery
        print("\n📂 Available Resources:")
        try:
            resources = await client.list_resources()
            if resources:
                print(f"Found {len(resources)} resources:")
                for resource in resources:
                    print(f"  • {resource.uri}: {resource.name}")
            else:
                print("  No resources available")
        except Exception as e:
            print(f"  No resources available: {e}")
        
        # Prompts discovery
        print("\n📝 Available Prompts:")
        try:
            prompts = await client.list_prompts()
            if prompts:
                print(f"Found {len(prompts)} prompts:")
                for prompt in prompts:
                    print(f"  • {prompt.name}: {prompt.description}")
            else:
                print("  No prompts available")
        except Exception as e:
            print(f"  No prompts available: {e}")
        
        # 4. Show MCP Tool Syntax and Parameters
        print("\n🔧 MCP TOOL SYNTAX & PARAMETERS:")
        print("=" * 40)
        for tool in tools:
            print(f"\n📄 Tool: {tool.name}")
            print(f"   Description: {tool.description}")
            if hasattr(tool, 'inputSchema') and tool.inputSchema:
                print(f"   Parameters Schema:")
                print(json.dumps(tool.inputSchema, indent=4))
            else:
                print("   Parameters: No schema available")
        
        # 5. Execute tools via MCP Protocol
        print("\n🛠️  EXECUTING MCP TOOLS:")
        print("=" * 30)
        
        print("\n🌤️  Calling get_weather tool...")
        weather_result = await client.call_tool("get_weather", {"city": "London"})
        print(f"Result: {weather_result.structured_content}")
        
        print("\n🧮 Calling calculate tool...")
        calc_result = await client.call_tool("calculate", {"operation": "multiply", "a": 15, "b": 4})
        print(f"Result: {calc_result.structured_content}")
        
        print("\n👤 Calling get_user_info tool...")
        user_result = await client.call_tool("get_user_info", {"user_id": 1})
        print(f"Result: {user_result.structured_content}")
        
        # 6. Advanced MCP Operations
        print("\n🔬 ADVANCED MCP OPERATIONS:")
        print("=" * 35)
        
        # Resource operations (if available)
        print("\n📂 Resource Operations:")
        try:
            # Try to read a resource
            resource_content = await client.read_resource("example://resource/1")
            print(f"   Resource content: {resource_content}")
        except Exception as e:
            print(f"   No resources to read (expected for this server): {e}")
        
        # Prompt operations (if available)
        print("\n📝 Prompt Operations:")
        try:
            # Try to get a prompt
            prompt_result = await client.get_prompt("welcome", {"name": "User"})
            print(f"   Prompt result: {prompt_result}")
        except Exception as e:
            print(f"   No prompts available (expected for this server): {e}")
        
        print("\n✅ Simple MCP client demo completed!")
        print("💡 Key insight: Non-AI applications can use MCP protocol for comprehensive server interaction")
        print("📝 Notice: MCP protocol provides automatic discovery of tools, resources, and prompts!")
        print("🔧 Demonstrated methods: ping(), get_server_info(), get_server_capabilities(),")
        print("   list_tools(), list_resources(), list_prompts(), call_tool(), read_resource(), get_prompt()")

# Handle event loop for notebook environment
try:
    await simple_mcp_client_demo()
except RuntimeError:
    import nest_asyncio
    nest_asyncio.apply()
    await simple_mcp_client_demo()

## 3. AI + MCP Client (Using Groq LLaMA)

This shows AI using MCP protocol to access tools - demonstrating AI + MCP integration.

In [ ]:
import os
from groq import Groq
from fastmcp import Client
import json

# Set up Groq API key (you'll need to set this)
GROQ_API_KEY = os.getenv('GROQ_API_KEY') or "your-groq-api-key-here"
groq_client = Groq(api_key=GROQ_API_KEY)

async def ai_mcp_client_demo():
    """Demonstrate AI application using MCP protocol with actual LLM calls"""
    print("🤖 AI + MCP CLIENT DEMO (Real LLM Integration)")
    print("=" * 50)
    
    # Connect to MCP server
    # MCP Client URL Options based on server transport:
    # - SSE (current):           "http://localhost:8765/sse"
    # - HTTP:                    "http://localhost:8765"
    # - Streamable HTTP:         "http://localhost:8765/stream"
    # - STDIO:                   Client("mcp_server.py") - simple file path syntax
    mcp_client = Client("http://localhost:8765/sse")
    async with mcp_client:
        # Get available tools from MCP server
        print("\n📋 AI discovering tools via MCP...")
        tools = await mcp_client.list_tools()
        available_tools = [tool.name for tool in tools]
        print(f"Available tools: {available_tools}")
        
        # Show tools with their MCP schemas
        print("\n🔍 AI analyzing MCP tool schemas:")
        tools_description = []
        for tool in tools:
            print(f"\n• {tool.name}:")
            print(f"  Description: {tool.description}")
            
            tool_info = f"- {tool.name}: {tool.description}"
            if hasattr(tool, 'inputSchema') and tool.inputSchema:
                properties = tool.inputSchema.get('properties', {})
                required = tool.inputSchema.get('required', [])
                print(f"  Parameters: {list(properties.keys())}")
                print(f"  Required: {required}")
                
                params_info = []
                for param, details in properties.items():
                    param_type = details.get('type', 'unknown')
                    is_required = " (required)" if param in required else " (optional)"
                    params_info.append(f"{param}: {param_type}{is_required}")
                tool_info += f" Parameters: {', '.join(params_info)}"
            else:
                print("  Parameters: No schema")
                tool_info += " Parameters: None"
            
            tools_description.append(tool_info)
        
        # User requests to process
        user_requests = [
            "What's the weather like in Tokyo?",
            "Calculate 25 divided by 5",
            "Get information for user ID 2"
        ]
        
        print("\n🎯 AI processing user requests using LLM + MCP tools:")
        
        for user_input in user_requests:
            print(f"\n👤 User: {user_input}")
            
            # Create prompt for LLM to choose the right tool
            tools_list = "\n".join(tools_description)
            prompt = f"""You are an AI assistant that can call tools to help users. 
                            Available tools:
                            {tools_list}

                            User query: "{user_input}"

                            Based on the user query, determine which tool to use and what parameters to provide.
                            Respond with ONLY a JSON object in this format:
                            {{
                                "tool_name": "tool_name_here",
                                "parameters": {{"param1": "value1", "param2": "value2"}}
                            }}

                            Do not include any other text in your response."""

            try:
                # Call LLM to decide which tool to use
                print("🧠 Asking LLM to choose the right tool...")
                
                if GROQ_API_KEY != "your-groq-api-key-here":
                    chat_completion = groq_client.chat.completions.create(
                        messages=[{"role": "user", "content": prompt}],
                        model="llama3-8b-8192",
                        temperature=0.1
                    )
                    
                    llm_response = chat_completion.choices[0].message.content.strip()
                    print(f"🧠 LLM response: {llm_response}")
                    
                    # Parse LLM response
                    try:
                        tool_choice = json.loads(llm_response)
                        tool_name = tool_choice["tool_name"]
                        parameters = tool_choice["parameters"]
                        
                        print(f"🤖 AI selected: {tool_name} with parameters {parameters}")
                        
                        # Call the chosen tool via MCP
                        result = await mcp_client.call_tool(tool_name, parameters)
                        print(f"   Result: {result.structured_content}")
                        
                    except json.JSONDecodeError:
                        print("❌ Failed to parse LLM response as JSON")
                else:
                    # Fallback simulation when no API key
                    print("⚠️  No Groq API key - simulating LLM response...")
                    if "weather" in user_input.lower():
                        tool_choice = {"tool_name": "get_weather", "parameters": {"city": "Tokyo"}}
                    elif "calculate" in user_input.lower() or "divided" in user_input.lower():
                        tool_choice = {"tool_name": "calculate", "parameters": {"operation": "divide", "a": 25, "b": 5}}
                    elif "user" in user_input.lower():
                        tool_choice = {"tool_name": "get_user_info", "parameters": {"user_id": 2}}
                    
                    print(f"🤖 Simulated AI choice: {tool_choice['tool_name']} with parameters {tool_choice['parameters']}")
                    result = await mcp_client.call_tool(tool_choice["tool_name"], tool_choice["parameters"])
                    print(f"   Result: {result.structured_content}")
                    
            except Exception as e:
                print(f"❌ Error in LLM call or tool execution: {e}")
        
        print("\n✅ AI + MCP demo completed!")
        print("💡 Key insight: LLM chooses tools dynamically based on user query and MCP tool schemas")
        print("📝 Notice: Same tools discovered via MCP, but LLM makes the intelligent choice!")

# Handle event loop for notebook environment
try:
    await ai_mcp_client_demo()
except RuntimeError:
    import nest_asyncio
    nest_asyncio.apply()
    await ai_mcp_client_demo()

## 4. Traditional Function Calling with AI

This demonstrates traditional function calling where we import the actual functions from the MCP server and use them directly (without the @mcp.tool decorator effect).

In [ ]:
import json
import inspect
from mcp_server import get_weather, calculate, get_user_info
from groq import Groq
import os

# Groq client for LLM calls
GROQ_API_KEY = os.getenv('GROQ_API_KEY') or "your-groq-api-key-here"
groq_client = Groq(api_key=GROQ_API_KEY)

def create_function_schema_from_mcp_tools():
    """Create function calling schemas dynamically from MCP server tools"""
    # Import the actual functions from mcp_server (same functions, used directly for function calling)
    mcp_functions = {
        "get_weather": get_weather,
        "calculate": calculate, 
        "get_user_info": get_user_info
    }
    
    schemas = []
    for func_name, func in mcp_functions.items():
        # Get function signature and docstring
        sig = inspect.signature(func)
        doc = func.__doc__ or f"Function {func_name}"
        
        # Build parameters schema from function signature
        properties = {}
        required = []
        
        for param_name, param in sig.parameters.items():
            param_type = "string"  # default
            if param.annotation == int:
                param_type = "integer"
            elif param.annotation == float:
                param_type = "number"
            elif param.annotation == str:
                param_type = "string"
            
            properties[param_name] = {
                "type": param_type,
                "description": f"The {param_name} parameter"
            }
            
            if param.default == inspect.Parameter.empty:
                required.append(param_name)
        
        schema = {
            "type": "function",
            "function": {
                "name": func_name,
                "description": doc.strip(),
                "parameters": {
                    "type": "object", 
                    "properties": properties,
                    "required": required
                }
            }
        }
        schemas.append(schema)
    
    return schemas, mcp_functions

def function_calling_demo():
    """Demonstrate traditional function calling with AI using actual MCP server functions and real LLM"""
    print("⚡ TRADITIONAL FUNCTION CALLING DEMO (Real LLM Integration)")
    print("=" * 65)
    
    # Generate schemas from actual MCP server functions
    function_schemas, available_functions = create_function_schema_from_mcp_tools()
    
    print(f"\n📋 Using actual functions from mcp_server.py:")
    for func_name in available_functions.keys():
        print(f"  • {func_name}")
    
    print(f"\n🔧 Generated {len(function_schemas)} function schemas dynamically")
    print("📝 Schemas created from function signatures & docstrings")
    
    # Show function calling schemas in detail
    print(f"\n📄 FUNCTION CALLING SCHEMAS:")
    print("=" * 35)
    for i, schema in enumerate(function_schemas, 1):
        print(f"\n{i}. Function: {schema['function']['name']}")
        print(f"   Description: {schema['function']['description']}")
        print(f"   Parameters:")
        properties = schema['function']['parameters']['properties']
        required = schema['function']['parameters']['required']
        for param_name, param_info in properties.items():
            req_mark = " (required)" if param_name in required else " (optional)"
            print(f"     • {param_name}: {param_info['type']}{req_mark}")
            print(f"       {param_info['description']}")
    
    # Show one complete example schema
    print(f"\n📋 Complete JSON Schema Example (get_weather):")
    print(json.dumps(function_schemas[0], indent=2))
    
    # Simulate user requests with LLM integration
    user_requests = [
        "What's the weather in New York?",
        "Add 10 and 15", 
        "Get user 3's information"
    ]
    
    print(f"\n🎯 EXECUTING FUNCTION CALLS WITH LLM:")
    print("=" * 40)
    
    for user_input in user_requests:
        print(f"\n👤 User: {user_input}")
        
        # Create prompt for LLM with function schemas
        functions_description = []
        for schema in function_schemas:
            func_info = schema['function']
            name = func_info['name']
            desc = func_info['description']
            params = func_info['parameters']['properties']
            required = func_info['parameters']['required']
            
            param_list = []
            for param, details in params.items():
                param_type = details['type']
                is_req = " (required)" if param in required else " (optional)"
                param_list.append(f"{param}: {param_type}{is_req}")
            
            functions_description.append(f"- {name}: {desc} | Parameters: {', '.join(param_list)}")
        
        functions_list = "\n".join(functions_description)
        
        prompt = f"""You are an AI assistant that can call functions to help users.
Available functions:
{functions_list}

User query: "{user_input}"

Based on the user query, determine which function to call and what parameters to provide.
Respond with ONLY a JSON object in this format:
{{
    "function_name": "function_name_here",
    "parameters": {{"param1": "value1", "param2": "value2"}}
}}

Do not include any other text in your response."""

        try:
            # Call LLM to decide which function to use
            print("🧠 Asking LLM to choose the right function...")
            
            if GROQ_API_KEY != "your-groq-api-key-here":
                chat_completion = groq_client.chat.completions.create(
                    messages=[{"role": "user", "content": prompt}],
                    model="llama3-8b-8192",
                    temperature=0.1
                )
                
                llm_response = chat_completion.choices[0].message.content.strip()
                print(f"🧠 LLM response: {llm_response}")
                
                # Parse LLM response
                try:
                    function_choice = json.loads(llm_response)
                    function_name = function_choice["function_name"]
                    parameters = function_choice["parameters"]
                    
                    print(f"🤖 AI selected: {function_name} with parameters {parameters}")
                    
                    # Call the chosen function directly
                    if function_name in available_functions:
                        if function_name == "get_weather":
                            result = available_functions[function_name](parameters["city"])
                        elif function_name == "calculate":
                            result = available_functions[function_name](
                                parameters["operation"], 
                                parameters["a"], 
                                parameters["b"]
                            )
                        elif function_name == "get_user_info":
                            result = available_functions[function_name](parameters["user_id"])
                        
                        print(f"   Result: {result}")
                    else:
                        print(f"❌ Unknown function: {function_name}")
                        
                except json.JSONDecodeError:
                    print("❌ Failed to parse LLM response as JSON")
            else:
                # Fallback simulation when no API key
                print("⚠️  No Groq API key - simulating LLM response...")
                if "weather" in user_input.lower():
                    city = "New York" if "new york" in user_input.lower() else "London"
                    function_choice = {"function_name": "get_weather", "parameters": {"city": city}}
                elif "add" in user_input.lower():
                    function_choice = {"function_name": "calculate", "parameters": {"operation": "add", "a": 10, "b": 15}}
                elif "user" in user_input.lower():
                    function_choice = {"function_name": "get_user_info", "parameters": {"user_id": 3}}
                
                print(f"🤖 Simulated AI choice: {function_choice['function_name']} with parameters {function_choice['parameters']}")
                
                # Execute the simulated choice
                if function_choice["function_name"] == "get_weather":
                    result = available_functions["get_weather"](function_choice["parameters"]["city"])
                elif function_choice["function_name"] == "calculate":
                    result = available_functions["calculate"](
                        function_choice["parameters"]["operation"],
                        function_choice["parameters"]["a"],
                        function_choice["parameters"]["b"]
                    )
                elif function_choice["function_name"] == "get_user_info":
                    result = available_functions["get_user_info"](function_choice["parameters"]["user_id"])
                
                print(f"   Result: {result}")
                
        except Exception as e:
            print(f"❌ Error in LLM call or function execution: {e}")
    
    print("\n✅ Function calling demo completed!")
    print("💡 Key insight: LLM chooses functions based on user query and function schemas")
    print("📝 Notice: Same functions from MCP server, but called directly with LLM guidance")
    print("🔄 LLM analyzes schemas → chooses function → app executes direct call")

# Run the demo
function_calling_demo()

## 5. Comparison Summary

Let's compare the three approaches we just demonstrated:

## 5. Comparison Summary

### Let's compare the three approaches we just demonstrated:

| Aspect               | MCP Server                   | Function Calling            |
|----------------------|------------------------------|-----------------------------|
| Protocol             | Standardized MCP protocol    | AI-specific implementation  |
| Communication        | Network-based (HTTP/SSE/WS)  | In-process direct calls     |
| Tool Discovery       | Dynamic via list_tools()     | Static JSON schema          |
| Schema Discovery     | Automatic via MCP protocol   | Manual introspection/coding |
| LLM Integration      | LLM + MCP tools list         | LLM + function schemas      |
| Decision Making      | LLM chooses from MCP tools   | LLM chooses from func schemas|
| Language Support     | Language agnostic            | Language dependent          |
| Scalability          | Multi-client, distributed    | Single application          |
| Coupling             | Loose - tools independent    | Tight - embedded functions  |
| Reusability          | High - any client can use    | Low - app-specific          |
| Use Cases            | Universal tool sharing       | AI-specific functionality   |
| Maintenance          | Centralized tool updates     | Per-app function updates    |
| Performance          | Network overhead             | Direct function calls       |
| Parameter Info       | Rich schema via protocol     | Function signatures only    |


### 🎯 KEY INSIGHTS

#### 🏆 MCP Advantages:
- Tool sharing across applications
- Standardized protocol with rich schemas
- Language-agnostic integration
- Network-based scalability
- NOT limited to AI use cases
- Dynamic tool discovery with full parameter info
- Automatic schema generation and validation
- LLM gets tools list from MCP server dynamically

#### 🏆 Function Calling Advantages:
- Lower latency (no network)
- Simpler implementation
- Better for single-app scenarios
- Direct function execution
- No network dependencies
- LLM works with static function schemas

### 🧠 LLM INTEGRATION COMPARISON

**🔄 MCP Flow:**
1. User query → App
2. App gets MCP tools list
3. User query + MCP tools → LLM
4. LLM chooses tool + parameters
5. App calls MCP tool
6. Result returned to user

**🔄 Function Calling Flow:**
1. User query → App
2. App has predefined function schemas
3. User query + function schemas → LLM
4. LLM chooses function + parameters
5. App calls function directly
6. Result returned to user

### 🌟 CONCLUSION
Both approaches use LLM for intelligent tool/function selection, but MCP provides dynamic tool discovery and universal sharing while Function Calling offers direct execution with lower latency.

**This demo showed:**
- SAME functions used in 3 different ways
- LLM decides which tool/function to use in both approaches
- MCP provides rich tool schemas automatically
- Non-AI applications can discover and use MCP tools
- AI applications can use MCP tools with LLM guidance
- Traditional function calling uses LLM with static schemas
- MCP enables universal tool sharing with dynamic discovery

**🔍 Schema & LLM Integration:**
- MCP: LLM analyzes dynamic tool list from server
- Function Calling: LLM analyzes static function schemas
- MCP: Rich parameter descriptions and validation
- Function Calling: Basic type information from introspection
- Both: LLM makes intelligent choices based on user intent


## 6. Cleanup

In [ ]:
# Stop the MCP server
print("🧹 CLEANING UP DEMO ENVIRONMENT")
print("=" * 35)
print("🛑 Stopping MCP server...")
server_process.terminate()
server_process.wait()
print("✅ MCP server stopped")

print("\n✅ Cleanup completed!")
print("\n📝 Demo Summary:")
print("  1. ✅ Started MCP server")
print("  2. ✅ Demonstrated simple MCP client (non-AI)")
print("  3. ✅ Showed AI + MCP integration")
print("  4. ✅ Compared with function calling using SAME functions")
print("  5. ✅ Generated schemas dynamically from MCP server functions")
print("  6. ✅ Cleaned up resources")

print("\n🎉 Demo completed successfully!")
print("\n🎯 Proven: MCP ≠ Function Calling")
print("💡 MCP enables universal tool sharing beyond AI!")